# Business Data Science | Fall 2019
## Problem Set 5
### Rutwick Bhawsar | Shivang Arya

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.formula.api import glm
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.cov_struct import (Exchangeable,
    Independence,Autoregressive)

from statsmodels.discrete.discrete_model import Poisson

from statsmodels.genmod.families import Poisson
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from patsy import dmatrices
from math import exp
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

### Problem 1. Question 1.
Simply loading the data in the "NYC stop and frisk.dat"

In [2]:
df=pd.read_csv("NYC_stop_and_frisk.dat",sep=" ")
df = df.iloc[4:,0:-4].copy()
df.reset_index(drop=True,inplace=True)
df.rename(columns={'stop': 'Past_Arrests','and': 'Total_Population', 'frisk': 'Stopped_Population','data' : 'Precinct', '(with' : "Ethnicity", 'noise' : 'Crime'}, inplace=True)

In [3]:
df

,Past_Arrests,Total_Population,Stopped_Population,Precinct,Ethnicity,Crime
0,75,1720,191,1,1,1
1,36,1720,57,1,1,2
2,74,1720,599,1,1,3
3,17,1720,133,1,1,4
4,37,1368,62,1,2,1
5,39,1368,27,1,2,2
6,23,1368,149,1,2,3
7,3,1368,57,1,2,4
8,26,23854,135,1,3,1
9,32,23854,16,1,3,2


### Problem 1. Question 2.

In [4]:
# Converting the DataFrame to Numeric. It was originaly stored in String.
df = df.apply(pd.to_numeric)

#Getting Total Population among the 75 precincts.
totalpop = df.iloc[:,0].sum()

#Getting Ethnic Population among the 75 precincts.
totalblack = df.loc[df['Ethnicity']==1].sum()[0]
totalhispanic = df.loc[df['Ethnicity']==2].sum()[0]
totalwhite = df.loc[df['Ethnicity']==3].sum()[0]

#Getting the fractions
fractionpopblack = (totalblack/totalpop)*100
fractionpopwhite = (totalwhite/totalpop)*100
fractionpophispanic = (totalhispanic/totalpop)*100

print()
print("Fraction of Black Population corresponding to Total Population:", fractionpopblack)
print("Fraction of Hispanic Population corresponding to Total Population:", fractionpophispanic)
print("Fraction of White Population corresponding to Total Population:", fractionpopwhite)

#Getting total stops.
totalstops = df.iloc[:,1].sum()

#Getting total stop as per ethnicity.
stopsblack = df.loc[df['Ethnicity']==1].sum()[1]
stopshispanic = df.loc[df['Ethnicity']==2].sum()[1]
stopswhite = df.loc[df['Ethnicity']==3].sum()[1]

#Getting the fractions.
fractionstopsblack = (stopsblack/totalstops)*100
fractionstopswhite = (stopswhite/totalstops)*100
fractionstopshispanic = (stopshispanic/totalstops)*100

print("\nFraction of Black Stops corresponding to Total Stops:", fractionstopsblack)
print("Fraction of Hispanic Stops corresponding to Total Stops:", fractionstopshispanic)
print("Fraction of White Stops corresponding to Total Stops:", fractionstopswhite)
print()


Fraction of Black Population corresponding to Total Population: 53.12966063004109
Fraction of Hispanic Population corresponding to Total Population: 33.95449703241516
Fraction of White Population corresponding to Total Population: 12.915842337543754

Fraction of Black Stops corresponding to Total Stops: 27.546911368656584
Fraction of Hispanic Stops corresponding to Total Stops: 25.574140350701068
Fraction of White Stops corresponding to Total Stops: 46.87894828064235



### Problem 1. Question 3.

In [5]:
#Controlling for Ethinicity.
#Getting dummy variable to one-hot-encode the ethnicity.

dummy = pd.get_dummies(df['Ethnicity'])
df['Black'] = dummy.loc[:,1]
df['Hispanic'] = dummy.loc[:,2]
df['White'] = dummy.loc[:,3]
df.head(5)

,Past_Arrests,Total_Population,Stopped_Population,Precinct,Ethnicity,Crime,Black,Hispanic,White
0,75,1720,191,1,1,1,1,0,0
1,36,1720,57,1,1,2,1,0,0
2,74,1720,599,1,1,3,1,0,0
3,17,1720,133,1,1,4,1,0,0
4,37,1368,62,1,2,1,0,1,0


In [6]:
#As taught in class, we are defining the Exposure input as the product of Past Arrests and Total Population

df['Exposure'] = df['Past_Arrests'] * df['Total_Population']

#Using a Poisson regression to model the number of stops, controlling for ethnicity 
#and using the number of past arrests as an exposure input.

xandy = """Stopped_Population ~ Exposure+Black+Hispanic+White"""
response,predictors = dmatrices(xandy, df, return_type='dataframe')
crime=sm.GLM(response,predictors,family = sm.families.Poisson()).fit()
print(crime.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:     Stopped_Population   No. Observations:                  900
Model:                            GLM   Df Residuals:                      896
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.2744e+05
Date:                Thu, 17 Oct 2019   Deviance:                   2.4890e+05
Time:                        22:13:32   Pearson chi2:                 3.56e+05
No. Iterations:                     6   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.0306      0.002   2201.684      0.000       4.027       4.034
Exposure    9.256e-09    7.1e-11    130.422      0.0

### Problem 1. Question 4. 

In [7]:
print("The Coefficients are:")
print(crime.params,"\n")
btow = exp(crime.params[2]-crime.params[4])
htow = exp(crime.params[3]-crime.params[4])
print("Ratio Black to White:",btow)
print("Ratio Hispanic to White:",htow)

print("\nExplaination in words:\nIn proportion to arrest rates of the previous year, people with Black Ethinicity are",btow,"times more likely to be stopped compared to people of White Ethinicity")
print("\nIn proportion to arrest rates of the previous year, people with Hispanic Ethinicity are",htow,"times more likely to be stopped compared to people of White Ethinicity")

The Coefficients are:
Intercept    4.030566e+00
Exposure     9.255785e-09
Black        1.884793e+00
Hispanic     1.433298e+00
White        7.124751e-01
dtype: float64 

Ratio Black to White: 3.229468643346766
Ratio Hispanic to White: 2.056124591391472

Explaination in words:
In proportion to arrest rates of the previous year, people with Black Ethinicity are 3.229468643346766 times more likely to be stopped compared to people of White Ethinicity

In proportion to arrest rates of the previous year, people with Hispanic Ethinicity are 2.056124591391472 times more likely to be stopped compared to people of White Ethinicity


### Problem 1. Question 5.

In [8]:
#Adding the 75 Precincts to our DataFrame
#Controlling for Precincts.
#Getting dummy variable to one-hot-encode the ethnicity.

dummy = pd.get_dummies(df['Precinct'])
for i in range(75):
    strr = 'Precinct' + str(i+1) 
    df[strr] = dummy.iloc[:,i]
df.head(5)

,Past_Arrests,Total_Population,Stopped_Population,Precinct,Ethnicity,Crime,Black,Hispanic,White,Exposure,...,Precinct66,Precinct67,Precinct68,Precinct69,Precinct70,Precinct71,Precinct72,Precinct73,Precinct74,Precinct75
0,75,1720,191,1,1,1,1,0,0,129000,...,0,0,0,0,0,0,0,0,0,0
1,36,1720,57,1,1,2,1,0,0,61920,...,0,0,0,0,0,0,0,0,0,0
2,74,1720,599,1,1,3,1,0,0,127280,...,0,0,0,0,0,0,0,0,0,0
3,17,1720,133,1,1,4,1,0,0,29240,...,0,0,0,0,0,0,0,0,0,0
4,37,1368,62,1,2,1,0,1,0,50616,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#Using a Poisson regression to model the number of stops, controlling for ethnicity and Precincts
#and using the number of past arrests as an exposure input.

xandy = """Stopped_Population ~ Exposure+Black+Hispanic+White+Precinct1+Precinct2+Precinct3+Precinct4+Precinct5+Precinct6+Precinct7+Precinct8+Precinct9+Precinct10+Precinct11+Precinct12+Precinct13+Precinct14+Precinct15+Precinct16+Precinct17+Precinct18+Precinct19+Precinct20+Precinct21+Precinct22+Precinct23+Precinct24+Precinct25+Precinct26+Precinct27+Precinct28+Precinct29+Precinct30+Precinct31+Precinct32+Precinct33+Precinct34+Precinct35+Precinct36+Precinct37+Precinct38+Precinct39+Precinct40+Precinct41+Precinct42+Precinct43+Precinct44+Precinct45+Precinct46+Precinct47+Precinct48+Precinct49+Precinct50+Precinct51+Precinct52+Precinct53+Precinct54+Precinct55+Precinct56+Precinct57+Precinct58+Precinct59+Precinct60+Precinct61+Precinct62+Precinct63+Precinct64+Precinct65+Precinct66+Precinct67+Precinct68+Precinct69+Precinct70+Precinct71+Precinct72+Precinct73+Precinct74+Precinct75"""
response,predictors = dmatrices(xandy, df, return_type='dataframe')
crime2=sm.GLM(response,predictors,family = sm.families.Poisson()).fit()
print(crime2.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:     Stopped_Population   No. Observations:                  900
Model:                            GLM   Df Residuals:                      822
Model Family:                 Poisson   Df Model:                           77
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -85581.
Date:                Thu, 17 Oct 2019   Deviance:                   1.6517e+05
Time:                        22:13:32   Pearson chi2:                 1.95e+05
No. Iterations:                     7   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.8595      0.002   1807.422      0.000       3.855       3.864
Exposure    5.609e-09   9.35e-11     59.965      0.0

### Problem 1. Question 6. 

In [10]:
print("The Coefficients for the first 10 columns are:")
print(crime2.params[:10],"\n")
btow = exp(crime2.params[2]-crime2.params[4])
htow = exp(crime2.params[3]-crime2.params[4])
print("Ratio Black to White:",btow)
print("Ratio Hispanic to White:",htow)

print("\nExplaination in words:\nIn proportion to arrest rates of the previous year, people with Black Ethinicity are",btow,"times more likely to be stopped compared to people of White Ethinicity")
print("\nIn proportion to arrest rates of the previous year, people with Hispanic Ethinicity are",htow,"times more likely to be stopped compared to people of White Ethinicity")

The Coefficients for the first 10 columns are:
Intercept    3.859538e+00
Exposure     5.608800e-09
Black        1.840570e+00
Hispanic     1.363402e+00
White        6.555660e-01
Precinct1   -3.305808e-01
Precinct2   -2.806229e-01
Precinct3    5.581489e-01
Precinct4   -2.516858e-01
Precinct5    8.321383e-01
dtype: float64 

Ratio Black to White: 3.2706991767746216
Ratio Hispanic to White: 2.029594998062876

Explaination in words:
In proportion to arrest rates of the previous year, people with Black Ethinicity are 3.2706991767746216 times more likely to be stopped compared to people of White Ethinicity

In proportion to arrest rates of the previous year, people with Hispanic Ethinicity are 2.029594998062876 times more likely to be stopped compared to people of White Ethinicity


## Problem 2.

In [11]:
df2=pd.read_csv("nba_cc_fake_data.csv",index_col=0)
df2.head()

,Comp,Height,Points,Salary
0,9.0,76.0,27.0,0.0
1,7.0,78.0,39.0,0.0
2,9.0,76.0,39.0,0.0
3,9.0,74.0,39.0,0.0
4,9.0,74.0,26.0,0.0


### Problem 2. Task 1

Explaination: Linear Regression works well while predicting continuous values. In our case we have the Y variable as a mixture of categorical and continuous values. Where 0 is the value when a player doesn't make it to the NBA and the actual salary if he does. The 0 and 1 are the categorical values and the Salary is the continuous value. Linear Regression will not predict well in this case. This is shown by our r2 value in Problem 2. Task 2.

Also another reason would be that 90% of our Y variable is 0 and 10% is a continuous variable. The large amount of disparity in the output can be considered a factor for the poor performance of the Linear Regression Model.

### Problem 2. Task 2

In [12]:
#Dividing the Data into training and testing
X = df2.iloc[:,0:3]
y = df2.iloc[:,3]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Running Linear Regression on it.
reg = LinearRegression().fit(X, y)

#Predicting for testing data.
y_pred = reg.predict(X_test)

In [13]:
#Seeing how well our model performed
#As you can see this model performs very poorly. 

r2_score(y_test.values, y_pred)

0.19176262013019696

### Problem 2. Task 3

In the first-half of the composite model, we calculate the probabilities of a player making it to the NBA
For this we have compared 3 different models 
and then used the one with the best scores to predict the probability on test data. 

Here we are making the Y variable a categorical variable. 
Where, if a player makes it to the NBA, y=1 else y=0

In [14]:
df_nba=df2.copy()
for i in df_nba.index:
    if(df_nba.loc[i,'Salary']>0):
        df_nba.at[i,'Salary']=1
df_nba.head(10)

,Comp,Height,Points,Salary
0,9.0,76.0,27.0,0.0
1,7.0,78.0,39.0,0.0
2,9.0,76.0,39.0,0.0
3,9.0,74.0,39.0,0.0
4,9.0,74.0,26.0,0.0
5,8.0,75.0,46.0,0.0
6,8.0,78.0,35.0,0.0
7,7.0,75.0,56.0,0.0
8,9.0,77.0,37.0,0.0
9,9.0,78.0,63.0,1.0


In [15]:
#Dividing the Data into training and testing
X = df_nba.iloc[:,0:3]
y = df_nba.iloc[:,3]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Logistic Regression Model

In [16]:
lr=LogisticRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print("The Accuracy of the Model is:",accuracy_score(y_test.values, y_pred))
print("The Area Under Curve is:",roc_auc_score(y_test.values, y_pred))

The Accuracy of the Model is: 0.944
The Area Under Curve is: 0.5758514068059764


C:\Users\rutwi\Anaconda_3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [17]:
y_pred = lr.predict_proba(X_test)
print("The probabilities of players from the test data making it to the NBA are:")
X_output = X_test.copy()

X_output['Probabilites'] = y_pred[:,1]
X_output['Y'] = y_test
X_output.sort_values(by = 'Points',ascending = False).head(20)

The probabilities of players from the test data making it to the NBA are:


,Comp,Height,Points,Probabilites,Y
5470,8.0,75.0,77.0,0.648807,1.0
9685,9.0,77.0,73.0,0.732855,1.0
4684,8.0,73.0,72.0,0.544294,0.0
7348,7.0,77.0,70.0,0.148855,0.0
1002,10.0,77.0,69.0,0.837970,1.0
6004,9.0,76.0,68.0,0.611507,1.0
222,8.0,78.0,67.0,0.250877,0.0
3646,10.0,75.0,67.0,0.833683,1.0
8722,10.0,78.0,67.0,0.777977,1.0
4692,7.0,77.0,67.0,0.104469,0.0


#### Gaussian Naive Bayes Model

In [18]:
gnb = GaussianNB()
gnb_pred = gnb.fit(X_train, y_train).predict(X_test)

In [19]:
gnb_pred_prob = gnb.predict_proba(X_test)
print("The probabilities of players from the test data making it to the NBA are:")
X_output = X_test.copy()

X_output['Probabilites'] = gnb_pred_prob[:,1]
X_output['Y'] = y_test
X_output.sort_values(by = 'Points',ascending = False).head(20)

The probabilities of players from the test data making it to the NBA are:


,Comp,Height,Points,Probabilites,Y
5470,8.0,75.0,77.0,0.084553,1.0
9685,9.0,77.0,73.0,0.591433,1.0
4684,8.0,73.0,72.0,0.034580,0.0
7348,7.0,77.0,70.0,0.006936,0.0
1002,10.0,77.0,69.0,0.791936,1.0
6004,9.0,76.0,68.0,0.453427,1.0
222,8.0,78.0,67.0,0.155450,0.0
3646,10.0,75.0,67.0,0.624512,1.0
8722,10.0,78.0,67.0,0.824742,1.0
4692,7.0,77.0,67.0,0.005953,0.0


In [20]:
print("The Accuracy of the Model is:",accuracy_score(y_test.values, gnb_pred))
print("The Area Under Curve is:",roc_auc_score(y_test.values, gnb_pred))

The Accuracy of the Model is: 0.952
The Area Under Curve is: 0.6187703147885063


#### Decision Tree Classifier Model

In [21]:
dtree = tree.DecisionTreeClassifier()
dtree_pred = dtree.fit(X_train,y_train).predict(X_test)

In [22]:
dtree_pred_prob = dtree.predict_proba(X_test)

print("The probabilities of players from the test data making it to the NBA are:")
X_output = X_test.copy()

X_output['Probabilites'] = dtree_pred_prob[:,1]
X_output['Y'] = y_test
X_output.sort_values(by = 'Points',ascending = False).head(20)

The probabilities of players from the test data making it to the NBA are:


,Comp,Height,Points,Probabilites,Y
5470,8.0,75.0,77.0,0.000000,1.0
9685,9.0,77.0,73.0,1.000000,1.0
4684,8.0,73.0,72.0,0.000000,0.0
7348,7.0,77.0,70.0,0.000000,0.0
1002,10.0,77.0,69.0,1.000000,1.0
6004,9.0,76.0,68.0,1.000000,1.0
222,8.0,78.0,67.0,0.000000,0.0
3646,10.0,75.0,67.0,0.750000,1.0
8722,10.0,78.0,67.0,1.000000,1.0
4692,7.0,77.0,67.0,0.000000,0.0


In [23]:
print("The Accuracy of the Model is:",accuracy_score(y_test.values, dtree_pred))
print("The Area Under Curve is:",roc_auc_score(y_test.values, dtree_pred))

The Accuracy of the Model is: 0.9545
The Area Under Curve is: 0.6819589283907829


### Problem 2. Task 3. Continue..

In the second-half of the composite model, we calculate the salary of a player after making it to the NBA
For this we have compared 2 different models 
and then used the one with the best scores to predict the probability on test data. 

Here we are taking the Y variable as a continuous variable. 
Where, when a player makes it to the NBA, y=salary else drop row. 

In [24]:
df_sal=df2.loc[df2['Salary']>0]
df_sal.head(10)

,Comp,Height,Points,Salary
9,9.0,78.0,63.0,496315.0
27,10.0,76.0,52.0,741265.0
30,8.0,78.0,62.0,292281.0
115,10.0,77.0,63.0,782210.0
117,8.0,76.0,50.0,259365.0
136,9.0,76.0,50.0,407135.0
147,10.0,78.0,59.0,709341.0
148,10.0,80.0,49.0,839476.0
150,10.0,78.0,47.0,956352.0
163,9.0,80.0,40.0,542106.0


In [25]:
#Dividing the Data into training and testing
X = df_sal.iloc[:,0:3]
y = df_sal.iloc[:,3]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
print("The R^2 score for Linear Regression is:",r2_score(y_test.values, y_pred))

regr = RandomForestRegressor(max_depth=10, random_state=0, n_estimators=500).fit(X_train,y_train)
rfpred = regr.predict(X_test)

print("The R^2 for Random Forest is:",r2_score(y_test.values, rfpred))

The R^2 score for Linear Regression is: 0.5415468455217292
The R^2 for Random Forest is: 0.473748814282526


### Problem 2. Task 4

Computing the expected NBA salary of a high school basketball player who is 6' 6" tall, is averaging 46 points per game, and is playing in the second most competitive league (comp = 9), according to your model.

In [27]:
#Predicting whether the player will get in the NBA or not using Decision Tree
#Predicting the salary of the player using Linear Regression
#Using the composite model to predict the salary based on the probability that he gets in.

print("Probability of the player getting into the NBA is: ",dtree.predict_proba(np.array([9,78,46]).reshape(1, -1))[0][1])
print("Expected Salary is:",dtree.predict_proba(np.array([9,78,46]).reshape(1, -1))[0][1]*reg.predict(np.array([9,78,46]).reshape(1, -1))[0])

Probability of the player getting into the NBA is:  0.07142857142857142
Expected Salary is: 34779.8733726227
